In [24]:
import osmnx as ox
import pandas as pd
from unidecode import unidecode
from geopy.distance import geodesic
from shapely.geometry import LineString
from fuzzywuzzy import fuzz

In [47]:
def get_line_midpoint(coords):
    """
    Get the midpoint coordinates (x, y) of a LINESTRING defined by a list of coordinates.
    
    Args:
        coords (list of tuples): List of (x, y) coordinates representing the LINESTRING.
        
    Returns:
        tuple: Tuple containing the x and y coordinates of the midpoint.
    """
    line = LineString(coords)
    midpoint = line.interpolate(0.5, normalized=True)
    return (midpoint.x, midpoint.y)

def encode_names(x):
    unicoded = unidecode(str(x))
    unicoded = unicoded.replace("poghots", "street")
    return unicoded

def get_top_matched_coordinate(df, query, debug = False):
    max_score = 0
    top_row = None

    for index, row in df.iterrows():
        score = fuzz.ratio(query.lower(), row['name_en'].lower())
        if score > max_score:
            max_score = score
            top_row = row
            
    if debug:
        return top_row
    else:
        return top_row["coordinates"]


In [28]:
place_name = "Yerevan, Armenia"
graph = ox.graph_from_place(
    place_name,
    network_type='{"walk"}',
    custom_filter='["name:en"]'
)

In [60]:
gdfs = ox.utils_graph.graph_to_gdfs(graph, edges = True)
streets = gdfs[1]
streets["name_am"] = streets["name"]
streets["name_en"] = streets["name"].apply(encode_names)
streets["name_en"] = streets["name_en"].str.replace("`", "")
streets["coordinates"] = streets["geometry"].apply(get_line_midpoint)
streets["coordinates"] = streets["coordinates"].apply(lambda coords: (coords[1], coords[0]))
streets = streets.reset_index()
streets = streets[["name_am", "name_en", "coordinates"]]
streets = streets.dropna()

In [61]:
streets.to_csv('streets.csv', index=False)

In [49]:
get_top_matched_coordinate(streets, "Davtashen 4 St", debug = True)

name_am            Դավթաշենի 4-րդ փողոց
name_en         Davtasheni 4-rd poghots
coordinates    (44.4790618, 40.2134186)
Name: 2497, dtype: object

0        (40.1906722427931, 44.46234708546167)
1       (40.19167216210699, 44.46253132416954)
2       (40.15532783191185, 44.45101440879486)
3       (40.159092487710595, 44.4779542538313)
4        (40.1528887108029, 44.50010169956805)
                         ...                  
8904         (40.211587300000005, 44.53160325)
8905          (40.2115894, 44.531686449999995)
8906                 (40.2122695, 44.53222975)
8907    (40.21303392770567, 44.53240499633234)
8908    (40.21239691932678, 44.53413149819361)
Name: coordinates, Length: 8897, dtype: object